# Prioritizarion Playground

experimentation prior to finalization of weight of each factor

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import percentileofscore

## Getting Data

In [2]:
clients = pd.read_csv('fake_CRM_data/clients.csv')
policies = pd.read_csv('fake_CRM_data/policies.csv')

In [3]:
clients.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on,updated_on
0,Robin Rodriguez,C001611,Personal,"369 Valentine Loaf Suite 986, New Danny, AZ 33230",+1-772-275-6711x91125,robin.rodriguez@brown-smith.com,B_US00787,S001819,Prospect,2005-09-27,2025-11-28
1,Seth Jennings,C002570,Commercial,"PSC 8605, Box 5178, APO AA 08295",+1-962-353-7147x05535,seth.jennings@henry-burgess.biz,B_US00725,S005192,Active,2005-08-10,2025-11-28
2,Veronica Miller,C003734,Personal,"22927 Young Inlet Apt. 303, Michaelville, TX 2...",376-996-0462,veronica.miller@vance.com,B_US00725,S009043,Inactive,2005-08-27,2025-11-28


In [4]:
policies.head(3)

,policy_id,client_id,line_of_business,issuing_carrier,issued_on,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct
0,P008414,C001611,"Pharmacist, hospital",Ortiz PLC Insurance Inc.,2005-09-27,2025-09-27,1164.561809,Direct Bill,Canceled,2025-11-28,17.694337
1,P007998,C002570,Veterinary surgeon,Smith and Sons Insurance Inc.,2005-08-10,2025-08-10,2175.399539,Agency Bill,New,2025-11-28,9.411621
2,P004008,C003734,Occupational hygienist,"Martinez, Coleman and Ford Insurance Inc.",2005-08-27,2025-08-27,931.708588,Insured,Canceled,2025-11-28,19.817233


## Preprocessing

### to Datetime

In [5]:
clients.issued_on = pd.to_datetime(clients.issued_on)
clients.updated_on = pd.to_datetime(clients.updated_on)

policies.expiry_date = pd.to_datetime(policies.expiry_date)
policies.last_updated = pd.to_datetime(policies.last_updated)
policies.issued_on = pd.to_datetime(policies.issued_on)

In [6]:
clients.issued_on.dtype

dtype('<M8[ns]')

In [7]:
clients.client_id.duplicated().sum()

np.int64(0)

## Getting Scores

### time to expiry

In [8]:
today = datetime(
    year=2025,
    month=11,
    day=28
)

In [9]:
merged = pd.merge(left=clients, right=policies, how='right', on='client_id')
merged.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on_x,updated_on,policy_id,line_of_business,issuing_carrier,issued_on_y,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct
0,Robin Rodriguez,C001611,Personal,"369 Valentine Loaf Suite 986, New Danny, AZ 33230",+1-772-275-6711x91125,robin.rodriguez@brown-smith.com,B_US00787,S001819,Prospect,2005-09-27,2025-11-28,P008414,"Pharmacist, hospital",Ortiz PLC Insurance Inc.,2005-09-27,2025-09-27,1164.561809,Direct Bill,Canceled,2025-11-28,17.694337
1,Seth Jennings,C002570,Commercial,"PSC 8605, Box 5178, APO AA 08295",+1-962-353-7147x05535,seth.jennings@henry-burgess.biz,B_US00725,S005192,Active,2005-08-10,2025-11-28,P007998,Veterinary surgeon,Smith and Sons Insurance Inc.,2005-08-10,2025-08-10,2175.399539,Agency Bill,New,2025-11-28,9.411621
2,Veronica Miller,C003734,Personal,"22927 Young Inlet Apt. 303, Michaelville, TX 2...",376-996-0462,veronica.miller@vance.com,B_US00725,S009043,Inactive,2005-08-27,2025-11-28,P004008,Occupational hygienist,"Martinez, Coleman and Ford Insurance Inc.",2005-08-27,2025-08-27,931.708588,Insured,Canceled,2025-11-28,19.817233


In [10]:
merged['_time_to_expiry_days'] = (today - merged.expiry_date).apply(lambda x: x.days)
merged['_time_to_expiry_days']

0       62
1      110
2       93
3       55
4      361
      ... 
995    191
996    271
997    314
998    191
999    256
Name: _time_to_expiry_days, Length: 1000, dtype: int64

more `time_to_expiry_score` score for less `time_to_expiry`

time_to_exiry_score = (100-percentile(time_to_expiry))/10 

score out of 10 that is a measure of how close the expiry is

In [11]:
get_time_to_expiry_score = lambda x: (100 - percentileofscore(
    a=merged['_time_to_expiry_days'],
    score=x
))/10
merged['_time_to_epxiry_score'] = merged['_time_to_expiry_days'].apply(get_time_to_expiry_score).head(3)

In [12]:
merged.head(3)

,client_name,client_id,account_type,address,phone_number,email,branch_id,servicing_broker_id,status,issued_on_x,updated_on,policy_id,line_of_business,issuing_carrier,issued_on_y,expiry_date,premium_amount,premium_bill_to,renewal_status,last_updated,broker_commision_pct,_time_to_expiry_days,_time_to_epxiry_score
0,Robin Rodriguez,C001611,Personal,"369 Valentine Loaf Suite 986, New Danny, AZ 33230",+1-772-275-6711x91125,robin.rodriguez@brown-smith.com,B_US00787,S001819,Prospect,2005-09-27,2025-11-28,P008414,"Pharmacist, hospital",Ortiz PLC Insurance Inc.,2005-09-27,2025-09-27,1164.561809,Direct Bill,Canceled,2025-11-28,17.694337,62,8.34
1,Seth Jennings,C002570,Commercial,"PSC 8605, Box 5178, APO AA 08295",+1-962-353-7147x05535,seth.jennings@henry-burgess.biz,B_US00725,S005192,Active,2005-08-10,2025-11-28,P007998,Veterinary surgeon,Smith and Sons Insurance Inc.,2005-08-10,2025-08-10,2175.399539,Agency Bill,New,2025-11-28,9.411621,110,7.02
2,Veronica Miller,C003734,Personal,"22927 Young Inlet Apt. 303, Michaelville, TX 2...",376-996-0462,veronica.miller@vance.com,B_US00725,S009043,Inactive,2005-08-27,2025-11-28,P004008,Occupational hygienist,"Martinez, Coleman and Ford Insurance Inc.",2005-08-27,2025-08-27,931.708588,Insured,Canceled,2025-11-28,19.817233,93,7.49


In [13]:
from prioritization import time_to_expiry_score, premium_at_risk_score

In [14]:
time_to_expiry_score(merged['_time_to_expiry_days'])

0      8.340
1      7.020
2      7.490
3      8.555
4      0.115
       ...  
995    4.625
996    2.345
997    1.475
998    4.625
999    2.790
Name: _time_to_expiry_days, Length: 1000, dtype: float64

### premium Score

premium score = percentile(premium)/10

higher the premium, higher the score

In [16]:
merged['_premium_score'] = premium_at_risk_score(merged['premium_amount'])
merged[['_premium_score', 'premium_amount']].head()

,_premium_score,premium_amount
0,7.26,1164.561809
1,8.23,2175.399539
2,7.00,931.708588
3,9.62,5551.185458
4,0.36,158.824583
